In [1]:
import os
import pandas as pd
demand_train_A = '../data/Dataset/demand_train_A.csv'
geo_topo = '../data/Dataset/geo_topo.csv'
inventory_info_A = '../data/Dataset/inventory_info_A.csv'
product_topo = '../data/Dataset/product_topo.csv'
weight_A = '../data/Dataset/weight_A.csv'

demand_train_A = pd.read_csv(demand_train_A)
geo_topo = pd.read_csv(geo_topo)
inventory_info_A = pd.read_csv(inventory_info_A)
product_topo = pd.read_csv(product_topo)
weight_A = pd.read_csv(weight_A)

demand_test_A = '../data/Dataset/demand_test_A.csv'
demand_test_A = pd.read_csv(demand_test_A)

dfs = [demand_train_A,geo_topo,inventory_info_A,product_topo,weight_A,demand_test_A]

for df in dfs:
    if 'Unnamed: 0' in df.columns:
        df.drop(columns='Unnamed: 0',inplace=True)
    if 'ts' in df.columns:
        df = df.sort_values(by='ts')

In [2]:
all_data =pd.concat([demand_train_A,demand_test_A])
all_data = all_data.sort_values(by='ts')
all_data = all_data.reset_index().drop(columns='index')

In [3]:
#真实值串14天
submission = demand_test_A
submission['yesterday_qty'] = submission.groupby('unit')['qty'].shift(1).fillna(method='ffill').reset_index().sort_index().set_index('index')

submission['diff_1'] = submission['qty'] - submission['yesterday_qty']
submission['qty'] = submission['diff_1']

submission['shift_14']=submission.groupby('unit')['qty'].shift(-14).fillna(0).reset_index().sort_index().set_index('index')
submission = submission[['unit','ts','shift_14']].rename(columns={'shift_14':'qty'})


#按照7天聚合
submission['dt'] = pd.to_datetime(submission['ts'])
submission['weekofyear'] = submission['dt'].dt.weekofyear
submission['year'] = submission['dt'].dt.year
submission_week = submission.copy()
submission_week = submission_week.groupby(['weekofyear','year','unit'],as_index=False).sum()
submission_week['sum_qty'] = submission_week['qty']
submission = pd.merge(submission_week,submission,on = ['weekofyear','year','unit'])
submission['dayofweek'] = submission['dt'].dt.dayofweek
submission = submission[submission['dayofweek']==0]
submission = submission[['unit','ts','sum_qty']].rename(columns={'sum_qty':'qty'})


init_inventory = inventory_info_A.set_index(['unit'])['qty'].to_dict()
def consume_init_inventory(arr,init_val):
    remain = init_val
    i = 0
    while remain>0 and i<len(arr):
        arr[i] = max(0,arr[i]-remain)
        remain -= arr[i]
        i+=1
    return arr

r = []
for i,group in submission.groupby('unit'):

    unit = group['unit'].values[0]
    init_val = init_inventory[unit]
    
    group = group.sort_values(by='ts')
    qty_list = group['qty'].values
    qty_list = consume_init_inventory(qty_list,init_val)
    group['qty'] = qty_list
    r.append(group)

submission = pd.concat(r)


In [7]:
submission['qty'] = submission['qty'].astype(int)

In [8]:
submission.head()

,unit,ts,qty
3792,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-08,0
8216,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-15,0
12640,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-22,0
17064,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-29,0
21488,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-04-05,0


In [9]:
import datetime

submission.to_csv(("../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"), index=False)